# Section: Securing Federated Learning

- Lesson 1: Trusted Aggregator
- Lesson 2: Intro to Additive Secret Sharing
- Lesson 3: Intro to Fixed Precision Encoding
- Lesson 4: Secret Sharing + Fixed Precision in PySyft
- Final Project: Federated Learning wtih Encrypted Gradient Aggregation

# Lesson: Federated Learning with a Trusted Aggregator

In the last section, we learned how to train a model on a distributed dataset using Federated Learning. In particular, the last project aggregated gradients directly from one data owner to another. 

However, while in some cases it could be ideal to do this, what would be even better is to be able to choose a neutral third party to perform the aggregation.

As it turns out, we can use the same tools we used previously to accomplish this.

# Project: Federated Learning with a Trusted Aggregator

In [4]:
pip install syft

In [0]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

In [0]:
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [7]:
bob.add_workers([alice, secure_worker])
alice.add_workers([bob, secure_worker])
secure_worker.add_workers([alice, bob])

W0824 14:07:27.478001 139786501089152 base.py:654] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0824 14:07:27.479735 139786501089152 base.py:654] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0824 14:07:27.481107 139786501089152 base.py:654] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior
W0824 14:07:27.482034 139786501089152 base.py:654] Worker secure_worker already exists. Replacing old worker which could cause                     unexpected behavior
W0824 14:07:27.483205 139786501089152 base.py:654] Worker alice already exists. Replacing old worker which could cause                     unexpected behavior
W0824 14:07:27.485067 139786501089152 base.py:654] Worker bob already exists. Replacing old worker which could cause                     unexpected behavior


<VirtualWorker id:secure_worker #objects:0>

In [0]:
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]] ,requires_grad=False)
target = th.tensor([[0],[0],[1],[1.]] , requires_grad=False)

In [0]:
bobs_data = data[0:2].send(bob)
bobs_target = target[0:2].send(bob)

In [0]:
alices_data = data[2:].send(alice)
alices_target = target[2:].send(alice)

In [0]:
model = nn.Linear(2,1)

In [12]:
for round_iter in range(10) :     
  
      bobs_model = model.copy().send(bob)
      alices_model = model.copy().send(alice)

      bobs_opt = optim.SGD(params=bobs_model.parameters(), lr=0.1)
      alices_opt = optim.SGD(params=alices_model.parameters(), lr=0.1)

      for i in range(10) :

            bobs_opt.zero_grad()
            bobs_pred = bobs_model(bobs_data)
            bobs_loss = ((bobs_pred - bobs_target) **2).sum()
            bobs_loss.backward()

            bobs_opt.step()
            bobs_loss = bobs_loss.get().data
            bobs_loss
      
            alices_opt.zero_grad()
            alices_pred = alices_model(alices_data)
            alices_loss = ((alices_pred - alices_target) **2).sum()
            alices_loss.backward()

            alices_opt.step()
            alices_loss = alices_loss.get().data
            alices_loss
      
      
      
      alices_model.move(secure_worker)
      bobs_model.move(secure_worker)      
      
     
      
      secure_worker.clear_objects()
      
      print("Bob:" + str(bobs_loss) + " Alice:" + str(alices_loss))

Bob:tensor(1.3614e-05) Alice:tensor(0.0131)
Bob:tensor(1.3614e-05) Alice:tensor(0.0131)
Bob:tensor(1.3614e-05) Alice:tensor(0.0131)
Bob:tensor(1.3614e-05) Alice:tensor(0.0131)
Bob:tensor(1.3614e-05) Alice:tensor(0.0131)
Bob:tensor(1.3614e-05) Alice:tensor(0.0131)
Bob:tensor(1.3614e-05) Alice:tensor(0.0131)
Bob:tensor(1.3614e-05) Alice:tensor(0.0131)
Bob:tensor(1.3614e-05) Alice:tensor(0.0131)
Bob:tensor(1.3614e-05) Alice:tensor(0.0131)


# Lesson: Intro to Additive Secret Sharing

While being able to have a trusted third party to perform the aggregation is certainly nice, in an ideal setting we wouldn't have to trust anyone at all. This is where Cryptography can provide an interesting alterantive. 

Specifically, we're going to be looking at a simple protocol for Secure Multi-Party Computation called Additive Secret Sharing. This protocol will allow multiple parties (of size 3 or more) to aggregate their gradients without the use of a trusted 3rd party to perform the aggregation. In other words, we can add 3 numbers together from 3 different people without anyone ever learning the inputs of any other actors.

Let's start by considering the number 5, which we'll put into a varible x

In [0]:
x = 5

Let's say we wanted to SHARE the ownership of this number between two people, Alice and Bob. We could split this number into two shares, 2, and 3, and give one to Alice and one to Bob

In [14]:
bob_x_share = 2
alice_x_share = 3

decrypted_x = bob_x_share + alice_x_share
decrypted_x

5

Note that neither Bob nor Alice know the value of x. They only know the value of their own SHARE of x. Thus, the true value of X is hidden (i.e., encrypted). 

The truly amazing thing, however, is that Alice and Bob can still compute using this value! They can perform arithmetic over the hidden value! Let's say Bob and Alice wanted to multiply this value by 2! If each of them multiplied their respective share by 2, then the hidden number between them is also multiplied! Check it out!

In [15]:
bob_x_share = 2 * 2
alice_x_share = 3 * 2

decrypted_x = bob_x_share + alice_x_share
decrypted_x

10

This even works for addition between two shared values!!

In [16]:
# encrypted "5"
bob_x_share = 2
alice_x_share = 3

# encrypted "7"
bob_y_share = 5
alice_y_share = 2

# encrypted 5 + 7
bob_z_share = bob_x_share + bob_y_share
alice_z_share = alice_x_share + alice_y_share

decrypted_z = bob_z_share + alice_z_share
decrypted_z

12

As you can see, we just added two numbers together while they were still encrypted!!!

One small tweak - notice that since all our numbers are positive, it's possible for each share to reveal a little bit of information about the hidden value, namely, it's always greater than the share. Thus, if Bob has a share "3" then he knows that the encrypted value is at least 3.

This would be quite bad, but can be solved through a simple fix. Decryption happens by summing all the shares together MODULUS some constant. I.e.

In [17]:
x = 5

Q = 23740629843760239486723

bob_x_share = 23552870267 # <- a random number
alice_x_share = Q - bob_x_share + x
alice_x_share

23740629843736686616461

In [18]:
(bob_x_share + alice_x_share) % Q

5

So now, as you can see, both shares are wildly larger than the number being shared, meaning that individual shares no longer leak this inforation. However, all the properties we discussed earlier still hold! (addition, encryption, decryption, etc.)

# Project: Build Methods for Encrypt, Decrypt, and Add 

In this project, you must take the lessons we learned in the last section and write general methods for encrypt, decrypt, and add. Store shares for a variable in a tuple like so.

In [0]:
x_share = (2,5,7)

Even though normally those shares would be distributed amongst several workers, you can store them in ordered tuples like this for now :)

In [0]:
import random

In [0]:
Q=23740629843760239486723

In [0]:
x= 5

In [0]:
def encrypt(x, n_shares=3) :
      
      shares = list()
      
      for i in range(n_shares - 1) :
           shares.append(random.randint(0,Q))
          
      final_share = Q - (sum(shares) %  Q) + x
      
      shares.append(final_share)
      
      return tuple(shares)
    

In [0]:
def decrypt(shares):
      return sum(shares) % Q

In [0]:
def add(a, b) :
     
      c= list()
      
      assert(len(a)  == len(b))
      
      for i in range(len(a)) :
           c.append((a[i] + b[i]) % Q)
          
      return tuple(c)    

In [26]:
decrypt(add(encrypt(5), encrypt(10)))

15

In [27]:
x = encrypt(5)
x

(8184938992920865048999, 3632988170551367062397, 11922702680288007375332)

In [28]:
y = encrypt(10)
y

(2626642703191711123736, 15602103524767753345206, 5511883615800775017791)

In [29]:
z = add(x,y)
z

(10811581696112576172735, 19235091695319120407603, 17434586296088782393123)

In [30]:
decrypt(z)

15

# Lesson: Intro to Fixed Precision Encoding

As you may remember, our goal is to aggregate gradients using this new Secret Sharing technique. However, the protocol we've just explored in the last section uses positive integers. However, our neural network weights are NOT integers. Instead, our weights are decimals (floating point numbers).

Not a huge deal! We just need to use a fixed precision encoding, which lets us do computation over decimal numbers using integers!

In [0]:
BASE=10
PRECISION=4
Q=23740629843760239486723

In [0]:
def encode(x_dec):
    return int((x_dec * (BASE ** PRECISION)) % Q)

def decode(x_fp):
    return (x_fp if x_fp <= Q/2 else x_fp - Q) / BASE**PRECISION

In [33]:
encode(0.5)

5000

In [34]:
decode(23740629843760239486723)

0.0

In [35]:
x = encrypt(encode(5.5))
y = encrypt(encode(2.3))
z = add(x,y)
decode(decrypt(z))

7.8

# Lesson: Secret Sharing + Fixed Precision in PySyft

While writing things from scratch is certainly educational, PySyft makes a great deal of this much easier for us through its abstractions.

In [0]:
bob = bob.clear_objects()
alice = alice.clear_objects()
secure_worker = secure_worker.clear_objects()

In [37]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

### Secret Sharing Using PySyft

We can share using the simple .share() method!

In [0]:
x = x.share(bob, alice, secure_worker)

In [39]:
bob._objects

{37759585635: tensor([1307486639978963936, 1772543521463849406,  755332784552035436,
          357352296837860548, 4558057761578889798])}

and as you can see, Bob now has one of the shares of x! Furthermore, we can still call addition in this state, and PySyft will automatically perform the remote execution for us!

In [0]:
y = x + x

In [41]:
y

(Wrapper)>[AdditiveSharingTensor]
	-> [PointerTensor | me:69153440530 -> bob:98296074170]
	-> [PointerTensor | me:73409921479 -> alice:27482987306]
	-> [PointerTensor | me:66221913917 -> secure_worker:37769418130]
	*crypto provider: me*

In [42]:
y.get()

tensor([ 2,  4,  6,  8, 10])

### Fixed Precision using PySyft

We can also convert a tensor to fixed precision using .fix_precision()

In [0]:
x = th.tensor([0.1,0.2,0.3])

In [44]:
x

tensor([0.1000, 0.2000, 0.3000])

In [0]:
x = x.fix_prec()

In [46]:
x.child.child

tensor([100, 200, 300])

In [0]:
y = x + x

In [48]:
y = y.float_prec()
y

tensor([0.2000, 0.4000, 0.6000])

### Shared Fixed Precision

And of course, we can combine the two!

In [0]:
x = th.tensor([0.1, 0.2, 0.3])

In [0]:
x = x.fix_prec().share(bob, alice, secure_worker)

In [0]:
y = x + x

In [52]:
y.get().float_prec()

tensor([0.2000, 0.4000, 0.6000])

Make sure to make the point that people can see the model averages in the clear.

In [0]:
x= th.tensor([0.1,0.2,0.3]).fix_prec().share(bob, alice, secure_worker)

In [54]:
x

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:65152331605 -> bob:14688615297]
	-> [PointerTensor | me:21721573050 -> alice:51531065930]
	-> [PointerTensor | me:20883209566 -> secure_worker:22125518730]
	*crypto provider: me*

In [0]:
y = x + x

In [56]:
y

(Wrapper)>FixedPrecisionTensor>[AdditiveSharingTensor]
	-> [PointerTensor | me:25479128076 -> bob:50158886607]
	-> [PointerTensor | me:53730434968 -> alice:12483556925]
	-> [PointerTensor | me:9077176005 -> secure_worker:22669538592]
	*crypto provider: me*

In [0]:
y = y.get().float_prec()

In [58]:
y

tensor([0.2000, 0.4000, 0.6000])

# Final Project: Federated Learning with Encrypted Gradient Aggregation

In [0]:
import torchvision.datasets as datasets


In [60]:
import syft as sy
hook = sy.TorchHook(torch)


NameError: ignored

In [0]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)

In [0]:
bob = sy.VirtualWorker(hook, id="bob")  
alice = sy.VirtualWorker(hook, id="alice")
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

In [0]:
bob.clear_objects
alice.clear_objects
secure_worker.clear_objects
compute_nodes = [bob, alice]

In [0]:
compute_nodes

In [0]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 10000
        self.epochs = 50
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

torch.manual_seed(args.seed)

In [0]:
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))]) 
mnist_trainset = datasets.MNIST(root='../data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(mnist_trainset, batch_size=args.test_batch_size, shuffle=True)


mnist_testset = datasets.MNIST(root='../data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=args.test_batch_

In [0]:
train_distributed_dataset = []

for batch_idx, (data,target) in enumerate(train_loader):
    data = data.send(compute_nodes[batch_idx % len(compute_nodes)])
    target = target.send(compute_nodes[batch_idx % len(compute_nodes)])
    train_distributed_dataset.append((data, target))

In [0]:
from torch import nn, optim
import torch.nn.functional as F

In [0]:
class Classifier(nn.Module):
    """
    Forward Convolutional Neural Network Architecture model
    
    """
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [0]:
model = Classifier()
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr)

In [0]:
for epoch in range(1, args.epochs + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_distributed_dataset): # iterate through each worker's dataset
        
        model.send(data.location) #send the model to the right location
        
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad() # 1) erase previous gradients (if they exist)
        output = model(data)  # 2) make a prediction
        loss = F.nll_loss(output, target)  # 3) calculate how much we missed
        loss.backward()  # 4) figure out which weights caused us to miss
        optimizer.step()  # 5) change those weights
        model.get()  # get the model back (with gradients)
        
        if batch_idx % args.log_interval == 0:
            loss = loss.get() #get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * data.shape[0], len(train_loader),
                       100. * batch_idx / len(train_loader), loss.item()))